In [1]:
import pickle

# Load meta dict

with open('../../PNC/AllSubjectsMeta.bin', 'rb') as f:
    meta = pickle.load(f)
    
print(len(list(meta.keys())))

941


In [13]:
import random

wratDict = dict()

with open('../../PNC/wrat.csv', 'r') as f:
    lines = f.readlines()[1:]
    for line in lines:
        line = line.strip().split(',')
        wratDict[line[0]] = {'raw': line[2], 'std': line[3]}

print(len(wratDict))
print(type(random.choice(list(wratDict))))

857
<class 'str'>


In [15]:
restBlacklist = [220,233,274,284,472]
emoidBlacklist = [180]

def loadSeries(prefix, para, idx):
    with open('{:}/{:}_fmri_power264/timeseries/{:}.bin'.format(prefix, para, idx), 'rb') as f:
        return pickle.load(f)

for key,val in meta.items():
    if 'rest' in val:
        meta[key]['restData'] = loadSeries('../../PNC', 'rest', meta[key]['rest'])
        if int(meta[key]['rest']) in restBlacklist:
            meta[key]['badRest'] = True
    if 'nback' in val:
        meta[key]['nbackData'] = loadSeries('../../PNC', 'nback', meta[key]['nback'])
    if 'emoid' in val:
        meta[key]['emoidData'] = loadSeries('../../PNC', 'emoid', meta[key]['emoid'])
        if int(meta[key]['emoid']) in emoidBlacklist:
            meta[key]['badEmoid'] = True
    if str(key) in wratDict:
        meta[key]['meta']['wrat'] = wratDict[str(key)]
        
print('Complete')

Complete


In [30]:
# Validate

nAllPara = 0
badRest = []
badEmoid = []
nWrat = 0
nWratAllPara = 0

# NaN Wrat

badWrat = [603056110671, 603317055381, 604847970437]

for key in badWrat:
    meta[key]['meta'].pop('wrat', None)

for key in meta:
    if 'restData' in meta[key] and 'nbackData' in meta[key] and 'emoidData' in meta[key]:
        nAllPara += 1
    if 'badRest' in meta[key]:
        badRest.append(meta[key]['rest'])
    if 'badEmoid' in meta[key]:
        badEmoid.append(meta[key]['emoid'])
    if 'wrat' in meta[key]['meta']:
#         print(meta[key]['meta']['wrat'])
#         if 'wrat' in meta[key]['meta'] and meta[key]['meta']['wrat']['raw'] == 'NaN':
#             print(key)
        nWrat += 1
        if 'rest' in meta[key] and 'nback' in meta[key] and 'emoid' in meta[key]:
            if 'badRest' not in meta[key] and 'badEmoid' not in meta[key]:
                nWratAllPara += 1
        
print(nAllPara)
print(badRest)
print(badEmoid)
print(nWrat)
print(nWratAllPara)

624
['220', '233', '274', '284', '472']
['180']
851
594


In [23]:
print(meta[603056110671]['meta'])

{'AgeInMonths': 205, 'Gender': 'M', 'Ethnicity': 'CAUCASIAN/WHITE', 'AgeGroupID': 3, 'AgeGroupEdge1': 180, 'AgeGroupEdge2': 216}


In [24]:
print(wratDict['603056110671'])

{'raw': 'NaN', 'std': 'NaN'}


In [65]:
import numpy as np

# Test rest

rest = []

for key in meta:
    if 'restData' in meta[key] and 'badRest' not in meta[key]:
        rest.append(meta[key]['restData'])
        
rest = np.stack([np.corrcoef(ts) for ts in rest])
print(rest.shape)

(873, 264, 264)


In [68]:
import torch

ages = []

for key in meta:
    if 'restData' in meta[key] and 'badRest' not in meta[key]:
        ages.append(meta[key]['meta']['AgeInMonths']/12)

ages = np.stack(ages)
        
perm = np.random.permutation(np.arange(len(ages)))
rest_train = torch.from_numpy(rest[perm[0:800]]).float().cuda()
rest_test = torch.from_numpy(rest[perm[800:]]).float().cuda()
ages_train = torch.from_numpy(ages[perm[0:800]]).float().cuda()
ages_test = torch.from_numpy(ages[perm[800:]]).float().cuda()

print('Complete')

Complete


In [69]:
w,_,_,_ = torch.linalg.lstsq(rest_train.reshape([-1,264*264]),ages_train)

y_train = rest_train.reshape([-1,264*264])@w
y_test = rest_test.reshape([-1,264*264])@w

print(torch.mean(torch.abs(y_train-ages_train)))
print(torch.mean(torch.abs(y_test-ages_test)))

tensor(5.9843e-06, device='cuda:0')
tensor(2.1117, device='cuda:0')


In [57]:
# Test nback

rest = []

for key in meta:
    if 'nbackData' in meta[key] and 'badNback' not in meta[key]:
        rest.append(meta[key]['nbackData'])
        
rest = np.stack([np.corrcoef(ts) for ts in rest])
print(rest.shape)

(907, 264, 264)


In [63]:
ages = []

for key in meta:
    if 'nbackData' in meta[key] and 'badNback' not in meta[key]:
        ages.append(meta[key]['meta']['AgeInMonths']/12)

ages = np.stack(ages)
        
perm = np.random.permutation(np.arange(len(ages)))
rest_train = torch.from_numpy(rest[perm[0:800]]).float().cuda()
rest_test = torch.from_numpy(rest[perm[800:]]).float().cuda()
ages_train = torch.from_numpy(ages[perm[0:800]]).float().cuda()
ages_test = torch.from_numpy(ages[perm[800:]]).float().cuda()

print('Complete')

Complete


In [64]:
w,_,_,_ = torch.linalg.lstsq(rest_train.reshape([-1,264*264]),ages_train)

y_train = rest_train.reshape([-1,264*264])@w
y_test = rest_test.reshape([-1,264*264])@w

print(torch.mean(torch.abs(y_train-ages_train)))
print(torch.mean(torch.abs(y_test-ages_test)))

tensor(7.8499e-06, device='cuda:0')
tensor(1.8825, device='cuda:0')


In [70]:
# Test emoid

rest = []

for key in meta:
    if 'emoidData' in meta[key] and 'badEmoid' not in meta[key]:
        rest.append(meta[key]['emoidData'])
        
rest = np.stack([np.corrcoef(ts) for ts in rest])
print(rest.shape)

(678, 264, 264)


In [71]:
ages = []

for key in meta:
    if 'emoidData' in meta[key] and 'badEmoid' not in meta[key]:
        ages.append(meta[key]['meta']['AgeInMonths']/12)

ages = np.stack(ages)
        
perm = np.random.permutation(np.arange(len(ages)))
rest_train = torch.from_numpy(rest[perm[0:600]]).float().cuda()
rest_test = torch.from_numpy(rest[perm[600:]]).float().cuda()
ages_train = torch.from_numpy(ages[perm[0:600]]).float().cuda()
ages_test = torch.from_numpy(ages[perm[600:]]).float().cuda()

print('Complete')

Complete


In [72]:
w,_,_,_ = torch.linalg.lstsq(rest_train.reshape([-1,264*264]),ages_train)

y_train = rest_train.reshape([-1,264*264])@w
y_test = rest_test.reshape([-1,264*264])@w

print(torch.mean(torch.abs(y_train-ages_train)))
print(torch.mean(torch.abs(y_test-ages_test)))

tensor(1.6349e-05, device='cuda:0')
tensor(1.6441, device='cuda:0')


In [74]:
# Save MegaMeta

with open('../../PNC/MegaMeta.bin', 'wb') as f:
    pickle.dump(meta, f)